# Import Libraries

In [7]:

from pyspark.ml.recommendation import ALS
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import StructType,StructField,IntegerType


# Constants : Modify as and when required!

In [8]:
# file paths
business_file="Sample_Datasets/montreal_business/part-00000-b5f251d0-79e6-47a8-a405-042eb7b7894e-c000.snappy.parquet"
reviews_file="Sample_Datasets/montreal_reviews/part-00000-f0e4463e-0ac9-402e-b995-734cbefc958e-c000.snappy.parquet"
users_file="Sample_Datasets/montreal_users/part-00000-a7d49d78-89a7-478f-a577-0efe02dca047-c000.snappy.parquet"


# Initialize spark session

In [9]:
app_name="Collaborative filtering for restaurant recommendation"

def init_spark():
    spark = SparkSession \
        .builder \
        .appName(app_name) \
        .getOrCreate()
    return spark


# Load Dataset in Apache Spark

In [10]:
spark=init_spark()
business_df = spark.read.parquet(business_file)
reviews_df=spark.read.parquet(reviews_file)
users_df=spark.read.parquet(users_file)

#Standard Deviation of the ratings

In [19]:
from pyspark.sql.functions import stddev,mean
std_dev = reviews_df.select(stddev('stars')).collect()[0][0]
avg = reviews_df.select(mean('stars')).collect()[0][0]

print(std_dev)
print(avg)


1.2240193031101316
3.890370817263149


## Selecting required features

In our project, we are only concerned with a subset of columns from the dataset, specifically those that are relevant to our goal of recommending restaurants in Montreal. Therefore, we extract the necessary features from the business_df table, including the id, name, stars, category. 
Similarly, we filter the reviews_df table to include only reviews for the selected restaurants by performing an inner join with business_df.

In [12]:
business_df = business_df.select("business_id","name", "stars", 
                                 "review_count", "address", "city", "state", "postal_code", "longitude", 
                                 "categories", "latitude").withColumnRenamed("stars", "stars_restaurant")
reviews_df = reviews_df.join(business_df, on='business_id', how='inner')

## Preparing Data for ALS: Convert String to index
Prior to initiating the modeling process, it is essential to transform all the relevant columns to integer type for compatibility with the ALS model from pyspark. The columns requiring conversion are the business_id and user_id. We accomplish this by leveraging the StringIndexer function, which we imported from pyspark.ml.feature.

In [13]:
indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in ['business_id', 'user_id']]
pipeline = Pipeline(stages=indexer)
transformed = pipeline.fit(reviews_df).transform(reviews_df)
transformed.select(['business_id', 'user_id','business_id_index', 'user_id_index','stars','categories'])

DataFrame[business_id: string, user_id: string, business_id_index: double, user_id_index: double, stars: double, categories: string]

## Spliting the dataset into training and testing subsets 

Setting a seed value as 3 to make randomsplit output deterministic

In [14]:
(training, test) = transformed.randomSplit([0.8, 0.2],3)

## Create ALS model
The Apache Spark library provides various parameters for the ALS (Alternating Least Squares) algorithm, including:

- rank: the number of latent factors used in the model (default value: 10).
- maxIter: the maximum number of iterations to run (default value: 10).
- regParam: the regularization parameter used in ALS (default value: 1.0).
- implicitPrefs: a boolean value that indicates whether to use the explicit feedback ALS variant or the one adapted for implicit feedback data (default value: false, which means using explicit feedback).
- alpha: a parameter that applies to the implicit feedback variant of ALS, determining the baseline confidence in preference observations (default value: 1.0).
- nonnegative: a boolean value that specifies whether to use nonnegative constraints for least squares (default value: false).

In [15]:
als=ALS(maxIter=5,
        regParam=0.09,
        rank=20,
        userCol="user_id_index",
        itemCol="business_id_index",
        ratingCol="stars",
        coldStartStrategy="drop",
        nonnegative=True)

model=als.fit(training)

## Evaluate RMSE

In [ ]:
evaluator=RegressionEvaluator(metricName="rmse",labelCol="stars",predictionCol="prediction")
predictions=model.transform(test)
rmse=evaluator.evaluate(predictions)
print("RMSE="+str(rmse))

## Tuning Hyper Parameters

In [ ]:
values=[0.08,0.09,0.1,0.15,0.2,0.25,0.3,0.4,0.45,0.5,0.5,0.6,0.7,0.8,0.9]
for rm in values:
    als = ALS(maxIter=20,regParam=rm,rank=25,userCol="user_id_index",itemCol="business_id_index",ratingCol="stars",coldStartStrategy="drop",nonnegative=True)
    model = als.fit(training)
    evaluator=RegressionEvaluator(metricName="rmse",labelCol="stars",predictionCol="prediction")
    predictions=model.transform(test)
    rmse=evaluator.evaluate(predictions)
    print("RMSE="+str(rmse))
    print(model.userFactors.count())
    print(model.itemFactors.count())
    

In [ ]:
# we are uisng 0.4 as the regularizatin parameter as it yields lowest RMSE

In [17]:
als = ALS(maxIter=20,regParam=0.4,rank=25,userCol="user_id_index",itemCol="business_id_index",ratingCol="stars",coldStartStrategy="drop",nonnegative=True)
model = als.fit(training)
evaluator=RegressionEvaluator(metricName="rmse",labelCol="stars",predictionCol="prediction")
predictions=model.transform(test)
rmse=evaluator.evaluate(predictions)
print("RMSE="+str(rmse))
print("Given ratings vs Predicted ratings")
predictions.select(['user_id', 'business_id', 'stars', 'prediction']).show()





RMSE=1.236113620077952
Given ratings vs Predicted ratings
+--------------------+--------------------+-----+----------+
|             user_id|         business_id|stars|prediction|
+--------------------+--------------------+-----+----------+
|ysCBsXWPB-LAiewVS...|-WLV_MlPGVy4M98Xv...|  4.0| 2.2992718|
|3AxAep0OMmy_Lgids...|1cRvhC7shC6UneMqR...|  5.0| 3.4202719|
|8ptxwJ0c2pY0aQ6Pq...|1cRvhC7shC6UneMqR...|  4.0|  3.191821|
|xEzPJcx5Ny6NLm0p9...|1cRvhC7shC6UneMqR...|  3.0| 3.6124904|
|Vav-GApWFEVD5wGyK...|3vVHoQg2rCSWOq3yV...|  3.0| 2.7110302|
|hVb9u3kjXVxGZYJ74...|3vVHoQg2rCSWOq3yV...|  4.0| 2.7092052|
|zIOxNinC6ofIwwGjO...|3vVHoQg2rCSWOq3yV...|  2.0|   2.21408|
|7kGxOoefe5H-HMxKI...|5KsvP10z9InBcI6hh...|  2.0|  1.802579|
|8jsRaddgwE26drgje...|5KsvP10z9InBcI6hh...|  1.0| 0.8365478|
|9mR8YvTA3tJJL3K9B...|5KsvP10z9InBcI6hh...|  1.0| 2.5298412|
|GpV7fg9-iadPX5HI5...|5KsvP10z9InBcI6hh...|  3.0|  2.029737|
|OP7nO0dIaYW0O6DYX...|5KsvP10z9InBcI6hh...|  2.0|  2.369381|
|OP7nO0dIaYW0O6DYX...|5KsvP

In [18]:
std_dev_model = predictions.selectExpr("sqrt(avg((stars-prediction)*(stars-prediction)))").collect()[0][0]
avg_rating_model = predictions.selectExpr("avg(stars)").collect()[0][0]

print("Standard deviation of predicted ratings: ", std_dev_model)
print("Mean rating: ", avg_rating_model)

Standard deviation of predicted ratings:  1.236113620077952
Mean rating:  3.8865761405816848


In [23]:
print("Difference Between Actual vs predicted")
print(f"Standard Deviation: \n from dataset: {std_dev} from model:{std_dev_model} Difference: {std_dev-std_dev_model}")
print(f"Average Rating: \n from dataset: {avg} from model:{avg_rating_model} Difference: {avg-avg_rating_model}")

Difference Between Actual vs predicted
Standard Deviation: 
 from dataset: 1.2240193031101316 from model:1.236113620077952 Difference: -0.012094316967820262
Average Rating: 
 from dataset: 3.890370817263149 from model:3.8865761405816848 Difference: 0.0037946766814642707


# Find Best Recommendations

In [ ]:
def get_recommendations():
    """
    Returns top recommendations for a user.

    Returns
    -------
    :py:class:`pyspark.sql.DataFrame`
    a DataFrame of (itemCol, recommendations), where recommendations are
    stored as an array of ('name','business_id', 'stars', 'categories') Rows.
    """
    test = model.recommendForAllUsers(10).filter(col('user_id_index')==30).select("recommendations").take(10)
    topRestaurants = []
    for item in test[0][0]:        
        topRestaurants.append(item.business_id_index)
    
    schema = StructType([StructField("business_id_index",IntegerType(),True)])
    restaurants = spark.createDataFrame(topRestaurants,IntegerType()).toDF("business_id_index")
    return restaurants

In [ ]:
def display_transformed_list():
    transformed\
    .select(['name', 'user_id', 'stars', 'categories'])\
    .filter(col('user_id_index')==30)\
    .show()

In [ ]:
def display_top10_recommendations(restaurants):
    """
    Displays the top 10 restaurant recommendations.
    """
    restaurants\
    .join(transformed, on = 'business_id_index', how = 'inner')\
    .select([ 'name','business_id', 'stars', 'categories'])\
    .drop_duplicates(subset=['name'])\
    .show(10)

## Display the Top Recommendations

In [ ]:


top10_recommendations = get_recommendations()
display_top10_recommendations(top10_recommendations)
